In [103]:
from importlib import reload
from dotenv import load_dotenv

load_dotenv(r"C:\Users\vireima\tie-dashboard\.env")

import os

# print(os.getenv("SEVERA_CLIENT_SCOPE"))

import pandas as pd
import altair as alt
import arrow

import src.severa.base_client
import src.severa.fetch
import src.severa.client
import src.database
from src.daterange import DateRange
import datetime

reload(src.severa.base_client)
reload(src.severa.fetch)
reload(src.severa.client)
reload(src.database)

span = DateRange(540)
alt.data_transformers.disable_max_rows()

# async with src.severa.fetch.Fetcher() as f:
#    df = await f.get_resource_allocations(span)

DataTransformerRegistry.enable('default')

In [146]:
async with src.severa.fetch.Fetcher() as f:
    #b = await f.get_realized_invoices(DateRange(-30))
    #h = await f.get_realized_hours(DateRange(-30))
    total = await f.get_total(DateRange(-30))

2023-06-12 16:30:53.985 | SUCCESS  | src.severa.base_client:get_with_retries:128 - HTTP/2 GET users [0]: Response (9) in 0.22s.
2023-06-12 16:30:54.536 | SUCCESS  | src.severa.base_client:get_with_retries:128 - HTTP/2 GET projects [0]: Response (100) in 0.53s.
2023-06-12 16:30:54.864 | SUCCESS  | src.severa.base_client:get_with_retries:128 - HTTP/2 GET projects [0]: Response (100) in 0.31s.
2023-06-12 16:30:55.161 | SUCCESS  | src.severa.base_client:get_with_retries:128 - HTTP/2 GET projects [0]: Response (100) in 0.28s.
2023-06-12 16:30:55.546 | SUCCESS  | src.severa.base_client:get_with_retries:128 - HTTP/2 GET projects [0]: Response (100) in 0.38s.
2023-06-12 16:30:55.918 | SUCCESS  | src.severa.base_client:get_with_retries:128 - HTTP/2 GET projects [0]: Response (100) in 0.34s.
2023-06-12 16:30:56.272 | SUCCESS  | src.severa.base_client:get_with_retries:128 - HTTP/2 GET projects [0]: Response (91) in 0.34s.
2023-06-12 16:30:56.509 | SUCCESS  | src.severa.base_client:get_with_retrie

In [147]:
total

,user-name,Laskutus,Sisäiset tunnit,Projektitunnit,Tunnit yhteensä,Poissaolot,Tuntikulu,Kulu,Kate,Kate/projektitunti,Kate/tunti,Laskutusaste
user,,,,,,,,,,,,
0d100bc1-8376-7ac5-7bbb-055cdc20497d,Johanna,4752.5,84.5,36.0,135.5,15.0,50.0,6775.0,-2022.5,-56.180556,-14.926199,0.265683
a7ac9445-5b48-6d3e-ae00-d5dd99ceb7c8,Topi,4762.5,38.5,77.5,136.5,20.5,43.0,5869.5,-1107.0,-14.283871,-8.10989,0.567766
78ddf0fe-9e39-5e5e-5096-a80409925de9,Joni,4940.9,0.0,4.5,109.5,105.0,52.0,5694.0,-753.1,-167.355556,-6.877626,0.041096
5cd93092-3385-3e89-003a-9fe6d03a424d,Panu,2500.0,0.0,20.0,20.0,0.0,95.0,1900.0,600.0,30.0,30.0,1.0
30c9d126-7cdc-53fc-3b05-7f98b25e3c98,Lari,7727.75,20.25,102.5,145.25,22.5,48.5,7044.625,683.125,6.664634,4.703098,0.70568
2441bfbf-78f2-0173-b0a1-8634ce167a0d,Margarita,7549.73,38.0,95.5,135.5,2.0,44.0,5962.0,1587.73,16.625445,11.717565,0.704797
eb8d73da-8c1c-7996-b085-332e777dff1a,Vladimirs,12502.1,34.5,81.0,130.5,15.0,48.5,6329.25,6172.85,76.208025,47.301533,0.62069
af48336f-0979-b971-4927-9b09d53a3cef,Miika,13603.3,39.5,43.0,90.0,7.5,56.0,5040.0,8563.3,199.146512,95.147778,0.477778
2f365dcd-6ada-f6d8-8d73-4f393745e513,Ville,19215.8,38.0,114.0,159.5,7.5,50.0,7975.0,11240.8,98.603509,70.475235,0.714734


In [107]:
t = arrow.utcnow()
span = DateRange(t.shift(days=-30), t.shift(days=30))

async with src.severa.client.Client() as f:
    hours = await f.fetch_hours(DateRange(0))

2023-06-15 18:37:09.607 | SUCCESS  | src.severa.base_client:get_with_retries:130 - HTTP/2 GET users [0]: Response (9) in 0.11s.
2023-06-15 18:37:09.689 | SUCCESS  | src.severa.base_client:get_with_retries:130 - HTTP/2 GET users/0d100bc1-8376-7ac5-7bbb-055cdc20497d/workhours [0]: Response (0) in 0.06s.
2023-06-15 18:37:09.692 | SUCCESS  | src.severa.base_client:get_with_retries:130 - HTTP/2 GET users/eb8d73da-8c1c-7996-b085-332e777dff1a/workhours [0]: Response (5) in 0.06s.
2023-06-15 18:37:09.732 | SUCCESS  | src.severa.base_client:get_with_retries:130 - HTTP/2 GET activities [0]: Response (0) in 0.12s.
2023-06-15 18:37:09.759 | SUCCESS  | src.severa.base_client:get_with_retries:130 - HTTP/2 GET users/af48336f-0979-b971-4927-9b09d53a3cef/workhours [0]: Response (1) in 0.12s.
2023-06-15 18:37:09.810 | SUCCESS  | src.severa.base_client:get_with_retries:130 - HTTP/2 GET users/5cd93092-3385-3e89-003a-9fe6d03a424d/workhours [0]: Response (0) in 0.12s.
2023-06-15 18:37:09.817 | SUCCESS  | sr

In [108]:
hours

,user,value,id,date,is_all_day,activity_type,project,productive,forecast_date
0,eb8d73da-8c1c-7996-b085-332e777dff1a,7.5,maximum,NaT,<NA>,<NA>,<NA>,<NA>,2023-06-15 00:00:00+00:00
1,af48336f-0979-b971-4927-9b09d53a3cef,7.5,maximum,NaT,<NA>,<NA>,<NA>,<NA>,2023-06-15 00:00:00+00:00
2,0d100bc1-8376-7ac5-7bbb-055cdc20497d,7.5,maximum,NaT,<NA>,<NA>,<NA>,<NA>,2023-06-15 00:00:00+00:00
3,78ddf0fe-9e39-5e5e-5096-a80409925de9,7.5,maximum,NaT,<NA>,<NA>,<NA>,<NA>,2023-06-15 00:00:00+00:00
4,5cd93092-3385-3e89-003a-9fe6d03a424d,1.5,maximum,NaT,<NA>,<NA>,<NA>,<NA>,2023-06-15 00:00:00+00:00
5,2f365dcd-6ada-f6d8-8d73-4f393745e513,7.5,maximum,NaT,<NA>,<NA>,<NA>,<NA>,2023-06-15 00:00:00+00:00
6,30c9d126-7cdc-53fc-3b05-7f98b25e3c98,7.5,maximum,NaT,<NA>,<NA>,<NA>,<NA>,2023-06-15 00:00:00+00:00
7,a7ac9445-5b48-6d3e-ae00-d5dd99ceb7c8,7.5,maximum,NaT,<NA>,<NA>,<NA>,<NA>,2023-06-15 00:00:00+00:00
8,2441bfbf-78f2-0173-b0a1-8634ce167a0d,7.5,maximum,NaT,<NA>,<NA>,<NA>,<NA>,2023-06-15 00:00:00+00:00
9,eb8d73da-8c1c-7996-b085-332e777dff1a,1.5,workhours,2023-06-15 00:00:00+00:00,<NA>,<NA>,3e63438e-7391-58ab-8507-db279fba9092,True,2023-06-15 00:00:00+00:00
